<a href="https://colab.research.google.com/github/smlra-kjsce/Introduction-to-VAEs-and-GANs/blob/master/CNN_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install torch torchvision

In [0]:
# make sure to enable GPU acceleration! == Done
device = 'cuda'

#!pip3 install torch torchvision
!pip install torchviz

#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from math import pi

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3520 sha256=9b65c072ff5bd155dc2cf01062a8bceda6985da4017eb1da7db69b69127d9b0f
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz


In [0]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=False
args['latent_dim']=2

In [0]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
'''test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)'''

data = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27085044.36it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 441334.14it/s]
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 7463757.10it/s]                            
8192it [00:00, 179712.32it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
class VAE(nn.Module):
  #This defines the structure of the NN.
  def __init__(self):
      super(VAE, self).__init__()
       
      self.encoder = nn.Sequential(
          nn.Conv2d(1, 32, kernel_size=3, stride = 1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'),
          nn.ReLU(),
          nn.Conv2d(32, 64, kernel_size=3, stride = 1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'),
          nn.MaxPool2d(2),
          nn.ReLU(),
          nn.Dropout(p=0.5),#,training=self.training),
          nn.Conv2d(64,64, kernel_size=3, stride = 1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'),
          nn.MaxPool2d(2),
          nn.ReLU(),
          nn.Dropout(p=0.5)#,training=self.training),
            
      )
      #latent space variables
      self.fc1 = nn.Linear(5*5*64,32)
      self.fc_mean = nn.Linear(32, args['latent_dim'])
      self.fc_var = nn.Linear(32, args['latent_dim'])
      self.fc2 = nn.Linear(args['latent_dim'], 64*5*5)
       
      self.decoder = nn.Sequential(  
          nn.ConvTranspose2d(64, 64, kernel_size=1, stride=1), 
          nn.ReLU(),
          nn.Dropout(p=0.5),#,training=self.training),
          nn.ConvTranspose2d(64,1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'),
          nn.Sigmoid()
          
      )
        
  def reparameterise(self, mu, var):
      epsilon = torch.randn(1000,2)#.cuda()
      z = mu + torch.exp(var)*epsilon
      return z
          
  def latent_dense(self,h):
      h = self.fc1(h)
      mu, var = self.fc_mean(h), self.fc_var(h)
      z = self.reparameterise(mu, var)
      return z, mu, var
        
  def encode(self, x):
      h = self.encoder(x).view(-1,5*5*64 )
      #print(h.size())
      z, mu, var = self.latent_dense(h)
      return z, mu, var
        
  def decode(self,z):
      z = self.fc2(z)
      #print(z.size())
      z = z.view(1000, 64, 5, 5)
      #print(z.size())
      z = self.decoder(z)
      print(z.size())
      return z
        
  def forward(self, x):
      z, mu, sigma = self.encode(x)
      z = self.decode(z)
      return z, mu, sigma

In [0]:
#loss fn
def VAELoss(x, z, mu, sigma):
  x = x.reshape(-1)
  z = z.reshape(-1)
  print(x.size())
  print(z.size())
  reconstr_loss = nn.BCELoss(x,z)
  print(x.size())
  kl_loss = -0.5 * torch.mean(1 + sigma - mu.pow(2) - np.exp(sigma), axis=-1)
  
  return torch.mean(reconstr_loss+kl_loss)
  

In [0]:
model = VAE()
model#.cuda()
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
def train(epoch):
    global model
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data = data.cuda()
        #Variables in Pytorch are differenciable. 
        data = Variable(data)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output, m, v = model(data)
        output = Variable(output)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        if args['cuda']:
            target = output.cuda()

        loss = VAELoss(data,output,m,v)
        #loss(data,output,m,v)
        print(loss)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
train(10)

torch.Size([1000, 1, 5, 5])
torch.Size([784000])
torch.Size([25000])


RuntimeError: ignored